# Ley de Zipf: Poda del vocabulario
En este notebook se calcula la cantidad de palabras que deberían estar en el 10%, 20% y 30% del vocabulario según la Ley de Zipf. Luego, se poda el vocabulario en esos porcentajes y se analiza:
- Qué porcentaje de las palabras podadas coincide con *stopwords*.
- Qué palabras podadas no son *stopwords* y si podrían ser importantes para la recuperación.

In [45]:
import numpy as np
from Tokenizador import Tokenizador
import matplotlib.pyplot as plt
from collections import Counter
import os

# Tokenizar y calcular frecuencias
tokenizador = Tokenizador(stopwords_path="stopwords.txt")
resultados = tokenizador.analizar_coleccion("datos/")

output_dir = "resultados/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
tokenizador.generar_archivos_salida(resultados, output_dir)

# Ordenar términos por frecuencia descendente
sorted_terms = sorted(
    [(termino, data["cf"]) for termino, data in resultados["terminos"].items()],
    key=lambda x: x[1],
    reverse=True
)

# Total de términos únicos
total_terminos = len(sorted_terms)
print(f"Total de términos únicos: {total_terminos}")

Total de términos únicos: 26645


## Calcular palabras esperadas en el 10%, 20% y 30% del vocabulario según la Ley de Zipf

In [46]:
ranks = list(range(1, len(sorted_terms) + 1))  # 1, 2, 3, ...
freqs = [freq for _, freq in sorted_terms]
log_ranks = np.log(ranks)
log_freqs = np.log(freqs)
coef = np.polyfit(x=log_ranks, y=log_freqs, deg=1)  # parámetros de la ecuación de zipf
print("Coeficientes de la recta log-log:", coef)

Coeficientes de la recta log-log: [-1.15691    11.44913022]


In [47]:
'''
Respuesta profe: si el vocabulario tiene una longitud de 1000 palabras y quisieras estimar cuántas palabras hay en las primeras 100 posiciones del ranking (10% del vocabulario), se puede sumar las frecuencias de cada palabra desde r = 1 hasta r=100, donde la frecuencia para cada posición viene dada por:

frec(r) = C.r^b (reemplazando C y b por los obtenidos en el punto 7)
'''

# Estimar la cantidad de palabras en el 10% del vocabulario utilizando la ley de Zipf
def estimar_frecuencia_zipf(r, C, b):
    return C * (r**b)
# Parámetros de la ley de Zipf
C = np.exp(coef[1])
b = coef[0]

print(f"Parámetros de la ley de Zipf: C = {C}, b = {b}")

for percent in [0.1, 0.2, 0.3]:
    n = int(total_terminos * percent)   # Número de términos a estimar (10% del vocabulario)
    # Frecuencias estimadas
    frecuencias_estimadas = [estimar_frecuencia_zipf(r, C, b) for r in range(1, n + 1)]
    print(f"Cantidad de tokens en el {percent*100:.0f}% del vocabulario (según Ley de Zipf): {sum(frecuencias_estimadas)}")
    print(f"Cantidad de tokens en el {percent*100:.0f}% del vocabulario (real): {sum(freqs[:n])}")

Parámetros de la ley de Zipf: C = 93819.70821221864, b = -1.1569099990102645
Cantidad de tokens en el 10% del vocabulario (según Ley de Zipf): 479703.0094362753
Cantidad de tokens en el 10% del vocabulario (real): 332707
Cantidad de tokens en el 20% del vocabulario (según Ley de Zipf): 497578.5236158122
Cantidad de tokens en el 20% del vocabulario (real): 351787
Cantidad de tokens en el 30% del vocabulario (según Ley de Zipf): 507164.886312633
Cantidad de tokens en el 30% del vocabulario (real): 361412


## Poda del vocabulario en 10%, 20% y 30%

In [48]:
def podar_vocabulario(terminos, porcentaje):
    n = int(len(terminos) * porcentaje)
    return dict(terminos[:n])

# Poda del vocabulario al 10%, 20% y 30%
for percent in [0.1, 0.2, 0.3]:
    terminos_podados = podar_vocabulario(sorted_terms, percent)

    # Verificar qué porcentaje de la poda coincide con palabras vacías. Extraiga las palabras podadas que no son *stopwords* y verifique si, a su criterio, pueden ser importantes para la recuperación.
    stopwords = set(tokenizador.stopwords)
    terminos_no_stopwords = {termino: data for termino, data in terminos_podados.items() if termino not in stopwords}
    print(f"\nPoda del vocabulario al {percent*100:.0f}%:")
    print(f"Palabras podadas: {len(terminos_podados)}")
    print(f"Palabras podadas que no son stopwords: {len(terminos_no_stopwords)}")
    print(f"Porcentaje de palabras podadas que no son stopwords: {len(terminos_no_stopwords) / len(terminos_podados) * 100:.2f}%")
    # for termino in terminos_no_stopwords:
    #     print(termino)



Poda del vocabulario al 10%:
Palabras podadas: 2664
Palabras podadas que no son stopwords: 2230
Porcentaje de palabras podadas que no son stopwords: 83.71%

Poda del vocabulario al 20%:
Palabras podadas: 5329
Palabras podadas que no son stopwords: 4809
Porcentaje de palabras podadas que no son stopwords: 90.24%

Poda del vocabulario al 30%:
Palabras podadas: 7993
Palabras podadas que no son stopwords: 7428
Porcentaje de palabras podadas que no son stopwords: 92.93%
